In [ ]:
# Setup
import sys
from pathlib import Path

notebook_dir = Path.cwd()
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from core import (
    FederalRevenueModel,
    IndividualIncomeTaxAssumptions,
    PayrollTaxAssumptions,
    CorporateIncomeTaxAssumptions,
    TaxReforms,
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Initialize revenue model
revenue_model = FederalRevenueModel()

print("✅ Tax modeling modules loaded!")

---

## 1. Federal Revenue Structure

| Tax Type | FY2024 Revenue | % of Total | Key Features |
|----------|---------------|------------|---------------|
| Individual Income | $2.4T | 49% | Progressive rates (10-37%) |
| Payroll | $1.7T | 35% | Flat rate on wages |
| Corporate | $0.5T | 10% | 21% flat rate |
| Other | $0.3T | 6% | Excise, estate, customs |

In [ ]:
# Run baseline revenue projection
print("🔄 Running baseline revenue projection...")

baseline = revenue_model.project_total_revenue(years=10, iterations=500)

# Display results
years = range(2024, 2034)
income_mean = baseline['income_tax'].mean(axis=1) / 1e12
payroll_mean = baseline['payroll_tax'].mean(axis=1) / 1e12
corporate_mean = baseline['corporate_tax'].mean(axis=1) / 1e12
total_mean = baseline['total'].mean(axis=1) / 1e12

print("\n📊 10-Year Revenue Projection (Baseline)")
print("=" * 60)
for i, year in enumerate(years):
    print(f"{year}: Total ${total_mean[i]:.2f}T (Income ${income_mean[i]:.2f}T | Payroll ${payroll_mean[i]:.2f}T | Corp ${corporate_mean[i]:.2f}T)")

In [ ]:
# Visualize revenue composition
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Stacked area chart
ax1 = axes[0]
ax1.stackplot(list(years), 
              [income_mean, payroll_mean, corporate_mean],
              labels=['Individual Income', 'Payroll', 'Corporate'],
              colors=['#2196F3', '#4CAF50', '#FF9800'],
              alpha=0.8)
ax1.set_xlabel('Year')
ax1.set_ylabel('Revenue ($ Trillions)')
ax1.set_title('Federal Revenue by Source', fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Growth rates
ax2 = axes[1]
total_growth = (total_mean[-1] / total_mean[0] - 1) * 100
income_growth = (income_mean[-1] / income_mean[0] - 1) * 100
payroll_growth = (payroll_mean[-1] / payroll_mean[0] - 1) * 100
corp_growth = (corporate_mean[-1] / corporate_mean[0] - 1) * 100

categories = ['Total', 'Individual', 'Payroll', 'Corporate']
growth_rates = [total_growth, income_growth, payroll_growth, corp_growth]
colors = ['#9C27B0', '#2196F3', '#4CAF50', '#FF9800']

bars = ax2.bar(categories, growth_rates, color=colors)
ax2.set_ylabel('10-Year Growth (%)')
ax2.set_title('Revenue Growth by Category', fontweight='bold')
ax2.bar_label(bars, fmt='%.1f%%', padding=5)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 2. Tax Elasticities: Behavioral Responses

When taxes change, people respond:

| Elasticity | Value | Meaning |
|-----------|-------|--------|
| Labor Supply | 0.1-0.3 | 10% tax ↑ → 1-3% work ↓ |
| Capital Gains | 0.5-1.0 | High sensitivity to rates |
| Corporate Location | 0.2-0.4 | Some mobility |
| Tax Evasion | 0.1-0.2 | Higher rates → more evasion |

### Laffer Curve
At some point, higher rates reduce revenue because behavioral responses dominate.

In [ ]:
# Illustrate Laffer Curve concept
fig, ax = plt.subplots(figsize=(10, 6))

tax_rates = np.linspace(0, 100, 100)
# Simplified Laffer curve (illustrative)
revenue = tax_rates * (100 - tax_rates) / 25  # Peaks at 50%

ax.plot(tax_rates, revenue, 'b-', linewidth=3)
ax.axvline(x=50, color='red', linestyle='--', label='Revenue-maximizing rate')
ax.fill_between(tax_rates, 0, revenue, alpha=0.3, color='blue')

# Mark current rates approximately
ax.scatter([37], [37 * (100-37) / 25], s=200, color='green', zorder=5, label='Top marginal rate (37%)')
ax.scatter([21], [21 * (100-21) / 25], s=200, color='orange', zorder=5, label='Corporate rate (21%)')

ax.set_xlabel('Tax Rate (%)', fontsize=12)
ax.set_ylabel('Tax Revenue (Illustrative)', fontsize=12)
ax.set_title('The Laffer Curve: Tax Rate vs Revenue', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Note: Actual optimal rates depend on many factors and are debated.")
print("   PoliSim incorporates empirical elasticity estimates from economic literature.")

---

## 3. Tax Reform Scenarios

Let's model some common reform proposals:

1. **Wealth Tax**: 2% annual tax on net worth > $50M
2. **Carbon Tax**: $50/ton CO2 emissions
3. **Rate Increases**: Raise top marginal rate to 45%
4. **Corporate Minimum**: 15% minimum tax on book income

In [ ]:
# Model simplified reform scenarios
print("📊 Tax Reform Revenue Estimates (Annual)")
print("=" * 60)

reforms = {
    'Wealth Tax (2% on >$50M)': {
        'low': 200,
        'mid': 300,
        'high': 400,
        'note': 'High uncertainty due to avoidance'
    },
    'Carbon Tax ($50/ton)': {
        'low': 150,
        'mid': 200,
        'high': 250,
        'note': 'Revenue declines as emissions fall'
    },
    'Top Rate to 45%': {
        'low': 50,
        'mid': 80,
        'high': 120,
        'note': 'Behavioral responses reduce gains'
    },
    'Corporate Minimum 15%': {
        'low': 100,
        'mid': 150,
        'high': 200,
        'note': 'Depends on enforcement'
    }
}

for reform, data in reforms.items():
    print(f"\n{reform}:")
    print(f"   Revenue: ${data['low']}-${data['high']}B/year (mid: ${data['mid']}B)")
    print(f"   Note: {data['note']}")

In [ ]:
# Visualize reform revenue potential
fig, ax = plt.subplots(figsize=(12, 6))

reform_names = list(reforms.keys())
mids = [reforms[r]['mid'] for r in reform_names]
lows = [reforms[r]['mid'] - reforms[r]['low'] for r in reform_names]
highs = [reforms[r]['high'] - reforms[r]['mid'] for r in reform_names]

colors = ['#E53935', '#4CAF50', '#2196F3', '#FF9800']

bars = ax.barh(reform_names, mids, color=colors, alpha=0.8)
ax.errorbar(mids, reform_names, xerr=[lows, highs], fmt='none', color='black', capsize=5)

ax.set_xlabel('Annual Revenue ($ Billions)', fontsize=12)
ax.set_title('Potential Revenue from Tax Reforms', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (bar, mid) in enumerate(zip(bars, mids)):
    ax.text(mid + 20, i, f'${mid}B', va='center', fontsize=10)

plt.tight_layout()
plt.show()

total_potential = sum(mids)
print(f"\n💰 Combined potential: ~${total_potential}B/year (${total_potential/1000:.1f}T over 10 years)")
print("   Note: Combining reforms may reduce total due to interactions.")

---

## 4. Try It Yourself: Custom Scenario

Design your own tax reform and estimate the revenue impact.

In [ ]:
# 🎮 EXPERIMENT: Custom tax scenario
# ==================================

# Run a baseline projection with modified assumptions
baseline_10yr = revenue_model.project_total_revenue(years=10, iterations=500)
baseline_total = baseline_10yr['total'].mean(axis=1).sum() / 1e12

# Simulate a simple revenue increase scenario
# (Adding 5% to all revenue categories)
revenue_increase_pct = 0.05  # Try: 0.03, 0.05, 0.10

modified_total = baseline_total * (1 + revenue_increase_pct)
additional_revenue = modified_total - baseline_total

print(f"📊 Custom Scenario Analysis")
print(f"=" * 50)
print(f"\nBaseline 10-year revenue: ${baseline_total:.2f}T")
print(f"Revenue increase: {revenue_increase_pct:.0%}")
print(f"Modified 10-year revenue: ${modified_total:.2f}T")
print(f"Additional revenue: ${additional_revenue:.2f}T")

---

## 🎓 What You've Learned

✅ Individual income tax is the largest federal revenue source (~49%)  
✅ Tax elasticities determine how behavior changes with rate changes  
✅ Reform proposals have wide uncertainty ranges  
✅ PoliSim can model revenue impacts with Monte Carlo uncertainty  

---

## 📚 Next Steps

| Notebook | Topic | Time |
|----------|-------|------|
| **07** | Policy Extraction | 30-45 min |
| **08** | API Integration | 30-45 min |
| **09** | Custom Policies | 60-90 min |

---

*Continue to Notebook 07: Policy Extraction* →